In [5]:
!pip install langchain-community
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 37.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 3.9 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 25.0
    Uninstalling packaging-25.0:
      Successfully uninstalled packaging-25.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.8.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
datasets 3.6.0 requires fsspec[http]<=2025.3.0,>=2023.1.0, but you have fsspec 2025.5.1 which is incompatible.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
pandas-gbq 0.29.1 requires google-api-core<3.0.0,>=2.10.2, but you have google-api-core 1.34.1 which is incompatible.
thinc 8.3

In [6]:
import os
import json
import pandas as pd
import torch
from tqdm import tqdm
from typing import List, Dict
from kaggle_secrets import UserSecretsClient
from huggingface_hub import login
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

user_secrets = UserSecretsClient()
hf_token = user_secrets.get_secret("hf_hub_token")
login(hf_token)

CSV_FILE = "/kaggle/input/quran-translation/Muhammad_Tahir-ul-Qadri_translation.csv"
QUESTIONS_FILE = "/kaggle/input/test-jsonl/test.jsonl"
OUTPUT_FILE = "/kaggle/working/rag_mistral_output.jsonl"
INDEX_PATH = "/kaggle/working/quran_faiss_index"

K = 4
MAX_TOKENS = 512
CHUNK_SIZE = 1764
CHUNK_OVERLAP = 0
LIMIT = 100
MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.1"
EMBEDDING_MODEL = "sentence-transformers/all-MiniLM-L6-v2"

2025-07-25 06:59:51.929795: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753426792.130266      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753426792.191038      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [7]:
def load_documents_from_csv(csv_path: str) -> List[Document]:
    df = pd.read_csv(csv_path)
    docs = []
    for _, row in df.iterrows():
        content = str(row["Ayat"]).strip()
        meta = {
            "surah": row["Surah"],
            "ayah": row["Ayat"],
            "verse": row["Verse"]
        }
        if content:
            docs.append(Document(page_content=content, metadata=meta))
    return docs


In [8]:
def build_or_load_faiss(docs: List[Document], embedder, force_recreate=False):
    if not force_recreate and os.path.exists(INDEX_PATH):
        return FAISS.load_local(INDEX_PATH, embedder,allow_dangerous_deserialization=True)

    splitter = RecursiveCharacterTextSplitter(chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP)
    chunks = splitter.split_documents(docs)
    db = FAISS.from_documents(chunks, embedder)
    db.save_local(INDEX_PATH)
    return db


In [9]:
def load_questions(path: str, limit: int = 100) -> List[Dict[str, str]]:
    questions = []
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            if len(questions) >= limit:
                break
            try:
                record = json.loads(line)
                question = record.get("Question") or record.get("question")
                if question and isinstance(question, str):
                    questions.append({"Question": question.strip()})
            except json.JSONDecodeError:
                continue
    return questions


In [10]:
def save_results(path: str, results: List[Dict]):
    with open(path, "w", encoding="utf-8") as f:
        for record in results:
            f.write(json.dumps(record, ensure_ascii=False) + "\n")

def generate_answer(pipe, question: str, context_chunks: List[Document]):
    context_str = "\n".join([doc.page_content for doc in context_chunks])
    prompt = f"""
<|user|>
Context:
{context_str}

Question: {question}
<|end|>
"""
    result = pipe(prompt, max_new_tokens=MAX_TOKENS, return_full_text=False, temperature=0.1)
    return result[0]['generated_text'].strip()


In [11]:
def main():
    print("🔁 Loading documents...")
    embedder = HuggingFaceEmbeddings(model_name=EMBEDDING_MODEL)
    documents = load_documents_from_csv(CSV_FILE)

    print("🔁 Building/loading FAISS index...")
    vector_db = build_or_load_faiss(documents, embedder)
    retriever = vector_db.as_retriever(search_type="similarity", search_kwargs={"k": K})

    print("🧠 Loading Mistral-7B model...")
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        torch_dtype=torch.bfloat16,
        attn_implementation="sdpa",
        device_map="auto"
    )
    pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

    print("📖 Loading questions...")
    questions = load_questions(QUESTIONS_FILE)

    results = []
    for idx, q in enumerate(tqdm(questions, desc="Running RAG")):
        question = q.get("Question", "").strip()
        if not question:
            continue

        retrieved_docs = retriever.get_relevant_documents(question)
        answer = generate_answer(pipe, question, retrieved_docs)

        reference = [
            f"Surah {doc.metadata.get('surah')}"
            for doc in retrieved_docs
        ]
        
        retrieved_texts = [f"{doc.metadata.get('verse')}" for doc in retrieved_docs]

        results.append({
            "Question": question,
            "Answer": answer,
            "Retrieved": retrieved_texts,
            "Reference": reference
        })

    print("💾 Saving results...")
    save_results(OUTPUT_FILE, results)
    print(f"✅ Done! Output saved to: {OUTPUT_FILE}")


In [12]:
def check():
    print("🔁 Loading documents...")
    embedder = HuggingFaceEmbeddings(model_name=EMBEDDING_MODEL)
    documents = load_documents_from_csv(CSV_FILE)

    print("🔁 Building/loading FAISS index...")
    vector_db = build_or_load_faiss(documents, embedder)
    retriever = vector_db.as_retriever(search_type="similarity", search_kwargs={"k": K})

    print("🧠 Loading Mistral-7B model...")
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        torch_dtype=torch.bfloat16,
        attn_implementation="sdpa",
        device_map="auto"
    )

    pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

    question = "Fasting in severe injury or medical emergency , ruling for muslims in ramadan month"

    retrieved_docs = retriever.get_relevant_documents(question)
    answer = generate_answer(pipe, question, retrieved_docs)
    reference = [
            f"Surah {doc.metadata.get('surah')}"
            for doc in retrieved_docs
        ]
    
    retrieved_texts = [f"{doc.metadata.get('verse')}" for doc in retrieved_docs]
    print(f"Answer: {answer}")
    print(f"\nRetrieved Doc: {retrieved_docs}")
    print(f"\nReference: {reference}")
    print(f"\nRetrieved Text: {retrieved_texts}")


In [13]:
# check()

In [14]:
# import shutil
# shutil.rmtree("/kaggle/working/quran_faiss_index/")

In [15]:
if __name__ == "__main__":
    main()

🔁 Loading documents...


/tmp/ipykernel_36/623437731.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedder = HuggingFaceEmbeddings(model_name=EMBEDDING_MODEL)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

Device set to use cuda:0


📖 Loading questions...


Running RAG:   0%|          | 0/100 [00:00<?, ?it/s]/tmp/ipykernel_36/623437731.py:29: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retrieved_docs = retriever.get_relevant_documents(question)
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Running RAG:  10%|█         | 10/100 [01:38<12:45,  8.50s/it]You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Running RAG: 100%|██████████| 100/100 [15:13<00:00,  9.14s/it]

💾 Saving results...
✅ Done! Output saved to: /kaggle/working/rag_mistral_output.jsonl
